In [1]:
push!(LOAD_PATH, "../")
using MyVQC
using LinearAlgebra

In [2]:
function rand_pure_state(n::Int64)
    r = rand(2^n)
    fac = sqrt(sum(r.^2))
    return r./fac
end

rand_pure_state (generic function with 1 method)

In [40]:
verify_normalization(state::Array{𝕋, 1}) where {𝕋<:Number} = abs(1 - sum(state.^2)) < 0.000001 ? true : false
#=
# deprecated!!!
function transform_mps!(state::Array{𝕋, 1}) where {𝕋<:Number}
    # check parameter and get properties
    !verify_normalization(state) && error("only pure quantum state can be transformed to mps")
    len = length(state)
    nqubits = Int(log(2, len))
    len != 2^nqubits && error("number of quantum state amplitude wrong")
    mps = initmps([])
    
    reshape_dims = 2
    for i in 1:(nqubits-1) # each iteration will generate one site
        # 1. transformation
        state = reshape(state, reshape_dims, Int(length(state) / reshape_dims))
        
        # 2. SVD
        F = svd(state)
        sqrt_s = sqrt(Diagonal(F.S))
        u = F.U * sqrt_s
        vt = sqrt_s * F.Vt
        
        # 3. another transformation
        reshape_dims = size(u)[2] * 2
        push!(mps.data, reshape(u, Int(size(u)[1]/2), 2, size(u)[2]))
        state = reshape(vt, length(vt)) # reshape to column vector
    end
    # 3. get the last site
    push!(mps.data, reshape(state, 2, 2, 1))
    return mps
end
=#

# arbitrary state隐含了tensor product，因此需要该函数首先将state转换，否则的话，结果的概率幅是反的
function __rev_state(state::Array{𝕋, 1}, nqubits::Int64) where {𝕋<:Number}
    # 1. get 0->2^nqubits-1
    # 2. change to bitstring, trim to nqubits' length and the reverse the order
    # 3. parse to Int and +1(julia's index start from 1)
    seq = [parse(Int, reverse(bitstring(i)[end-nqubits+1:end]), base=2)+1 for i in 0:(2^nqubits-1)]
    return [state[x] for x in seq]
end

function transform_mps2!(state::Array{𝕋, 1}) where {𝕋<:Number}
    # check parameter and get properties
    !verify_normalization(state) && error("only pure quantum state can be transformed to mps")
    len = length(state)
    nqubits = Int(log(2, len))
    len != 2^nqubits && error("number of quantum state amplitude wrong")
    mps = initmps([])
    
    state = __rev_state(state, nqubits)
    
    reshape_dims = 2
    for i in 1:nqubits # each iteration will generate one site
        # 1. transformation
        state = reshape(state, reshape_dims, Int(length(state) / reshape_dims))
        
        # 2. SVD
        F = svd(state)
        u = F.U
        #vt = Diagonal(F.S) * F.Vt
        vt = contract(Diagonal(F.S), F.Vt, (2,), (1,))
        
        # 3. another transformation
        reshape_dims = size(u)[2] * 2
        push!(mps.data, reshape(u, Int(size(u)[1]/2), 2, size(u)[2]))
        state = reshape(vt, length(vt)) # reshape to column vector
    end
    return mps
end


transform_mps2! (generic function with 1 method)

In [41]:
#=
    This section test for transform_mps!'s (Part I)
=#
# generate random pure state
Φ₁ = rand_pure_state(4)
Φ₂ = rand_pure_state(5)
Φ₃ = rand_pure_state(6)

mps₁ = transform_mps2!(Φ₁)
mps₂ = transform_mps2!(Φ₂)
mps₃ = transform_mps2!(Φ₃)
println("test1")
for each in mps₁.data
    println(size(each))
end

println("test2")
for each in mps₂.data
    println(size(each))
end

println("test3")
for each in mps₃.data
    println(size(each))
end

test1
(1, 2, 2)
(2, 2, 4)
(4, 2, 2)
(2, 2, 1)
test2
(1, 2, 2)
(2, 2, 4)
(4, 2, 4)
(4, 2, 2)
(2, 2, 1)
test3
(1, 2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 4)
(4, 2, 2)
(2, 2, 1)


In [44]:
#=
    This section test for transform_mps! (Part I)
=#
Φ = (1/sqrt(2)) * [0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0]
mps = transform_mps2!(Φ)
projection_all(mps)


projection for |ϕ>=|0000> == 0.0
projection for |ϕ>=|0001> == 0.0
projection for |ϕ>=|0010> == 0.0
projection for |ϕ>=|0011> == 0.7071067811865475
projection for |ϕ>=|0100> == 0.0
projection for |ϕ>=|0101> == 0.7071067811865475
projection for |ϕ>=|0110> == 0.0
projection for |ϕ>=|0111> == 0.0
projection for |ϕ>=|1000> == 0.0
projection for |ϕ>=|1001> == 0.0
projection for |ϕ>=|1010> == 0.0
projection for |ϕ>=|1011> == 0.0
projection for |ϕ>=|1100> == 0.0
projection for |ϕ>=|1101> == 0.0
projection for |ϕ>=|1110> == 0.0
projection for |ϕ>=|1111> == 0.0


In [46]:
#=
    This section test for transform_mps! (Part II)
=#
Φ = rand_pure_state(5)
for each in Φ
    println(each)
end
mps = transform_mps2!(Φ)
projection_all(mps)

0.25047085320439527
0.16906313382168758
0.2143865814491001
0.22620502890394767
0.21417783719898714
0.16450148995118488
0.13784947900089867
0.21024785863940249
0.22474065975589763
0.1681201276174252
0.17436739297575177
0.21050992402607585
0.04210808701375906
0.02789872209447038
0.16497593958277526
0.07360940726543955
0.19518429330017137
0.26136873545972733
0.0257092900920897
0.0062097466487569565
0.22926025709711065
0.18684799403352745
0.09496533738686404
0.1580739284301175
0.11970561924843484
0.033756711302044363
0.20023066144074506
0.13517518021927447
0.3002853722162093
0.26083322069514464
0.15280595459121044
0.052510596397955724

projection for |ϕ>=|00000> == 0.2504708532043955
projection for |ϕ>=|00001> == 0.1690631338216878
projection for |ϕ>=|00010> == 0.2143865814491004
projection for |ϕ>=|00011> == 0.22620502890394786
projection for |ϕ>=|00100> == 0.2141778371989874
projection for |ϕ>=|00101> == 0.16450148995118508
projection for |ϕ>=|00110> == 0.13784947900089892
projection for

In [47]:
Φ = rand_pure_state(3)
for each in Φ
    println(each)
end
mps = transform_mps2!(Φ)
projection_all(mps)

0.43174724589706776
0.04489467021616848
0.08994046981151702
0.5276753187114307
0.2888480882858795
0.20111175925961777
0.6246918440498834
0.10454279658590837

projection for |ϕ>=|000> == 0.4317472458970678
projection for |ϕ>=|001> == 0.044894670216168685
projection for |ϕ>=|010> == 0.0899404698115174
projection for |ϕ>=|011> == 0.5276753187114306
projection for |ϕ>=|100> == 0.28884808828587955
projection for |ϕ>=|101> == 0.20111175925961777
projection for |ϕ>=|110> == 0.6246918440498834
projection for |ϕ>=|111> == 0.1045427965859085
